In [440]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
from sklearn.preprocessing import StandardScaler
from plotly import express as px, graph_objects as go
from IPython.display import display, HTML
import plotly.io as pio
from pingouin import cronbach_alpha


csv_path = "C:/Users/dilos/Documents/coding proyects/py/muutivariados/Costumer Satisfacción Case Study/customer_satisfaction_data.csv"

data = pd.read_csv(csv_path)

## 1. Data Loading

In [441]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customer_id                 3400 non-null   object 
 1   quarter                     3400 non-null   object 
 2   survey_date                 3400 non-null   object 
 3   account_manager_responsive  3384 non-null   float64
 4   billing_accuracy            3387 non-null   float64
 5   budget_control              3383 non-null   float64
 6   change_management           3388 non-null   float64
 7   communication_clarity       3384 non-null   float64
 8   competitive_pricing         3385 non-null   float64
 9   cost_transparency           3390 non-null   float64
 10  documentation_help          3386 non-null   float64
 11  executive_access            3390 non-null   float64
 12  innovation_solutions        3390 non-null   float64
 13  long_term_partnership       3386 

In [442]:
data.describe()

,account_manager_responsive,billing_accuracy,budget_control,change_management,communication_clarity,competitive_pricing,cost_transparency,documentation_help,executive_access,innovation_solutions,...,technical_expertise,timeline_adherence,training_quality,trust_reliability,value_for_money,overall_satisfaction,nps_score,renewal_likelihood,revenue_growth_pct,referrals_generated
count,3384.000000,3387.000000,3383.000000,3388.000000,3384.000000,3385.000000,3390.000000,3386.000000,3390.000000,3390.000000,...,3391.000000,3392.000000,3383.000000,3378.000000,3385.000000,3400.000000,3400.000000,3400.000000,3400.000000,3400.000000
mean,4.115248,4.101270,4.100798,4.109504,4.092494,4.079468,4.100885,4.079445,4.113569,4.105310,...,4.115305,4.110554,4.072421,4.100355,4.091581,4.125588,6.169118,3.070294,6.072176,1.615588
std,0.970303,0.962109,0.983397,0.977813,0.964956,0.979341,0.981008,0.979800,0.969478,0.985496,...,0.974591,0.981026,0.998707,0.961346,0.975105,0.817824,1.777549,0.761728,8.315453,1.510634
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,-24.700000,0.000000
25%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,3.000000,...,4.000000,3.000000,3.000000,3.000000,3.000000,4.000000,5.000000,3.000000,0.300000,0.000000
50%,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,6.000000,3.000000,6.150000,1.000000
75%,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,7.000000,4.000000,11.700000,2.000000
max,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,...,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,10.000000,5.000000,40.100000,9.000000


In [443]:
pd.unique(data["overall_satisfaction"])

array([4, 3, 5, 6, 2, 7, 1])

In [444]:
out_vars = ["customer_id","quarter","survey_date"]
outcome_vars = ["overall_satisfaction","nps_score","renewal_likelihood","revenue_growth_pct","referrals_generated"]
variables = data.drop(out_vars,axis=1).dropna()
variables.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3235 entries, 0 to 3399
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   account_manager_responsive  3235 non-null   float64
 1   billing_accuracy            3235 non-null   float64
 2   budget_control              3235 non-null   float64
 3   change_management           3235 non-null   float64
 4   communication_clarity       3235 non-null   float64
 5   competitive_pricing         3235 non-null   float64
 6   cost_transparency           3235 non-null   float64
 7   documentation_help          3235 non-null   float64
 8   executive_access            3235 non-null   float64
 9   innovation_solutions        3235 non-null   float64
 10  long_term_partnership       3235 non-null   float64
 11  problem_solving             3235 non-null   float64
 12  project_management          3235 non-null   float64
 13  quality_deliverables        3235 non-n

## 2. Plots and EDA

In [445]:
corr_matrix = variables.corr()

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.index,
    colorscale='RdBu',
    zmin=-1,
    zmax=1,
    text=corr_matrix.round(2).values,
    hoverongaps = False,
))

fig.update_layout(
    title='Correlación de las Variables',
    xaxis_title="Variables",
    yaxis_title="Variables",
    height=700
)

display(HTML(pio.to_html(fig, full_html=True, include_plotlyjs='cdn')))

# 3. Data Standardization

In [446]:
variables_standarized = StandardScaler().fit_transform(variables.drop(outcome_vars,axis=1))
var_names = variables.drop(outcome_vars,axis=1).columns

# 4. Factor Analysis Assumptions Testing

In [447]:
chi_square_value, p_value = calculate_bartlett_sphericity(variables_standarized)
kmo_all, kmo_model = calculate_kmo(variables_standarized)

print("Factor Analysis Assumptions Testing:")
print("\nBartlett's Test of Sphericity:")
print(f"  Chi-square statistic: {chi_square_value:.3f}")
print(f"  p-value: {p_value:.6f}")
if p_value < 0.05:
    print("   Significant - variables are sufficiently correlated for FA")
else:
    print("   Not significant - FA may not be appropriate")

print("\nKaiser-Meyer-Olkin (KMO) Test:")
print(f"  Overall Measure of Sampling Adequacy: {kmo_model:.3f}")
if kmo_model > 0.9:
    adequacy = "Excellent"
elif kmo_model > 0.8:
    adequacy = "Good"
elif kmo_model > 0.6:
    adequacy = "Acceptable"
else:
    adequacy = "Unacceptable"
print(f"  Interpretation: {adequacy} sampling adequacy")

Factor Analysis Assumptions Testing:

Bartlett's Test of Sphericity:
  Chi-square statistic: 33163.378
  p-value: 0.000000
   Significant - variables are sufficiently correlated for FA

Kaiser-Meyer-Olkin (KMO) Test:
  Overall Measure of Sampling Adequacy: 0.959
  Interpretation: Excellent sampling adequacy


In [448]:
print("\nIndividual Variable Sampling Adequacy:")
for i, var_name in enumerate(var_names):
    msa_value = kmo_all[i]
    print(f"  {var_name}: {msa_value:.3f}")

# Flag any problematic variables
low_msa_vars = [
    var_name for i, var_name in enumerate(var_names) if kmo_all[i] < 0.6
]
if low_msa_vars:
    print(f"Variables with low MSA (<0.6): {low_msa_vars}")
else:
    print("All variables show adequate sampling adequacy (MSA >= 0.6)")


Individual Variable Sampling Adequacy:
  account_manager_responsive: 0.965
  billing_accuracy: 0.933
  budget_control: 0.967
  change_management: 0.966
  communication_clarity: 0.963
  competitive_pricing: 0.941
  cost_transparency: 0.942
  documentation_help: 0.954
  executive_access: 0.961
  innovation_solutions: 0.958
  long_term_partnership: 0.958
  problem_solving: 0.959
  project_management: 0.962
  quality_deliverables: 0.966
  roi_demonstration: 0.942
  support_responsiveness: 0.955
  system_integration: 0.961
  technical_documentation: 0.962
  technical_expertise: 0.962
  timeline_adherence: 0.965
  training_quality: 0.955
  trust_reliability: 0.959
  value_for_money: 0.952
All variables show adequate sampling adequacy (MSA >= 0.6)


In [449]:
alpha_value, ci = cronbach_alpha(data=pd.DataFrame(variables_standarized,columns=var_names))

print(f"Cronbach's Alpha: {alpha_value:.3f}")
print(f"95% Confidence Interval: {ci}")

# Interpretation:
if alpha_value >= 0.70:
    print("The scale has acceptable to excellent internal consistency.")
else:
    print("The scale has questionable or poor internal consistency.")

Cronbach's Alpha: 0.921
95% Confidence Interval: [0.917 0.925]
The scale has acceptable to excellent internal consistency.


# 5. Factor Extraction with Principal Axis Factoring

In [450]:
n_factors = 5 # Based on theoretical expectation of quantitative, verbal, and interpersonal factors

fa_unrotated = FactorAnalyzer(n_factors=n_factors, rotation=None, method="principal")
fa_unrotated.fit(variables_standarized)

# Verify successful extraction
if fa_unrotated.loadings_ is None:
    print("Factor extraction failed - no loadings produced")
    sys.exit(1)

print(f"Factor Analysis Results ({n_factors} factors extracted):")
eigenvalues_fa_unrotated = fa_unrotated.get_eigenvalues()[0]
print(f"Eigenvalues: {np.round(eigenvalues_fa_unrotated[:n_factors], 3)}")

Factor Analysis Results (5 factors extracted):
Eigenvalues: [8.747 1.776 1.425 1.204 1.073]


c:\Users\dilos\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [451]:
communalities = fa_unrotated.get_communalities()
uniquenesses = 1 - communalities

print("\nVariance Decomposition (Communalities and Uniquenesses):")
for i, var_name in enumerate(var_names):
    h2 = communalities[i]
    u2 = uniquenesses[i]
    print(f"  {var_name}: h² = {h2:.3f}, u² = {u2:.3f}")

# Analyze overall variance structure
factor_variance = np.sum(communalities)
total_variance = len(var_names)  # For standardized data
variance_explained_fa = factor_variance / total_variance

print("\nOverall Variance Analysis:")
print(f"Total standardized variance: {total_variance:.1f}")
print(f"Common variance (sum h²): {factor_variance:.3f}")
print(f"Proportion of variance explained by factors: {variance_explained_fa:.1%}")
print(f"Average communality: {np.mean(communalities):.3f}")


Variance Decomposition (Communalities and Uniquenesses):
  account_manager_responsive: h² = 0.608, u² = 0.392
  billing_accuracy: h² = 0.541, u² = 0.459
  budget_control: h² = 0.677, u² = 0.323
  change_management: h² = 0.673, u² = 0.327
  communication_clarity: h² = 0.620, u² = 0.380
  competitive_pricing: h² = 0.505, u² = 0.495
  cost_transparency: h² = 0.506, u² = 0.494
  documentation_help: h² = 0.508, u² = 0.492
  executive_access: h² = 0.626, u² = 0.374
  innovation_solutions: h² = 0.741, u² = 0.259
  long_term_partnership: h² = 0.623, u² = 0.377
  problem_solving: h² = 0.734, u² = 0.266
  project_management: h² = 0.698, u² = 0.302
  quality_deliverables: h² = 0.684, u² = 0.316
  roi_demonstration: h² = 0.511, u² = 0.489
  support_responsiveness: h² = 0.492, u² = 0.508
  system_integration: h² = 0.733, u² = 0.267
  technical_documentation: h² = 0.723, u² = 0.277
  technical_expertise: h² = 0.716, u² = 0.284
  timeline_adherence: h² = 0.683, u² = 0.317
  training_quality: h² = 0.

# 6. Inspect and Interpret Unrotated Factors

In [452]:
# Extract the unrotated loadings from the fitted model
loadings_unrotated = fa_unrotated.loadings_

# Create a detailed DataFrame of unrotated loadings
unrotated_loadings_df = pd.DataFrame(
    loadings_unrotated,
    columns=[f"Factor {i+1}" for i in range(n_factors)],
    index=var_names
)

print("\n" + "="*70)
print("UNROTATED FACTOR LOADINGS ANALYSIS")
print("="*70)

print("\nUnrotated Factor Loadings Matrix:")
print(unrotated_loadings_df.round(3))
print()


UNROTATED FACTOR LOADINGS ANALYSIS

Unrotated Factor Loadings Matrix:
                            Factor 1  Factor 2  Factor 3  Factor 4  Factor 5
account_manager_responsive     0.640    -0.076     0.404    -0.083    -0.151
billing_accuracy               0.465     0.536    -0.010     0.114    -0.158
budget_control                 0.720     0.039    -0.135    -0.313     0.201
change_management              0.711     0.048    -0.159    -0.331     0.172
communication_clarity          0.651    -0.076     0.420    -0.063    -0.104
competitive_pricing            0.450     0.517    -0.078     0.122    -0.118
cost_transparency              0.463     0.509    -0.057     0.152    -0.083
documentation_help             0.364    -0.020     0.158     0.315     0.501
executive_access               0.641    -0.069     0.432    -0.062    -0.139
innovation_solutions           0.724    -0.304    -0.243     0.209    -0.146
long_term_partnership          0.628    -0.095     0.445    -0.093    -0.112
probl

# 7. Factor Rotation

In [453]:
# Apply Varimax rotation
fa_rotated = FactorAnalyzer(n_factors=n_factors, rotation="varimax", method="principal")
fa_rotated.fit(variables_standarized)

loadings_rotated = fa_rotated.loadings_

# Safety check for rotation success
if loadings_rotated is None:
    print("Varimax rotation failed")
    sys.exit(1)

# Display rotated loadings
rotated_loadings_df = pd.DataFrame(
    loadings_rotated,
    columns=[f"Factor {i+1}" for i in range(n_factors)],
    index=var_names
)
eigenvalues_fa_rotated = fa_rotated.get_eigenvalues()[0]
print("\nVarimax Rotated Factor Loadings:")
print(rotated_loadings_df.round(3))


Varimax Rotated Factor Loadings:
                            Factor 1  Factor 2  Factor 3  Factor 4  Factor 5
account_manager_responsive     0.213     0.152     0.702     0.193     0.100
billing_accuracy               0.078     0.700     0.165     0.132     0.026
budget_control                 0.245     0.191     0.238     0.711     0.134
change_management              0.249     0.199     0.225     0.715     0.094
communication_clarity          0.204     0.149     0.702     0.199     0.153
competitive_pricing            0.101     0.678     0.095     0.156     0.042
cost_transparency              0.103     0.674     0.103     0.149     0.094
documentation_help             0.091     0.078     0.104     0.124     0.684
executive_access               0.201     0.158     0.716     0.176     0.127
innovation_solutions           0.779     0.133     0.213     0.229     0.135
long_term_partnership          0.181     0.116     0.721     0.196     0.133
problem_solving                0.773     0

c:\Users\dilos\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [454]:
loading_threshold = 0.4

print(f"\nFactor Structure Analysis (threshold = {loading_threshold}):")
for factor_idx in range(n_factors):
    factor_name = f"Factor {factor_idx + 1}"
    salient_vars = []

    for var_idx, var_name in enumerate(var_names):
        loading = loadings_rotated[var_idx, factor_idx]
        if abs(loading) > loading_threshold:
            salient_vars.append(f"{var_name} ({loading:+.3f})")

    print(
        f"  {factor_name}: {', '.join(salient_vars) if salient_vars else 'No salient loadings'}"
    )

factor_traductor = {}
for i,new_name in enumerate(["Tecnical_and_Product","Value_and_Money","Relationship_and_Trust","Project_Execution","Support_and_Training"]):
    factor_traductor.update({"Factor"+str(i):new_name})


Factor Structure Analysis (threshold = 0.4):
  Factor 1: innovation_solutions (+0.779), problem_solving (+0.773), system_integration (+0.766), technical_documentation (+0.759), technical_expertise (+0.757)
  Factor 2: billing_accuracy (+0.700), competitive_pricing (+0.678), cost_transparency (+0.674), roi_demonstration (+0.680), value_for_money (+0.642)
  Factor 3: account_manager_responsive (+0.702), communication_clarity (+0.702), executive_access (+0.716), long_term_partnership (+0.721), trust_reliability (+0.727)
  Factor 4: budget_control (+0.711), change_management (+0.715), project_management (+0.735), quality_deliverables (+0.724), timeline_adherence (+0.719)
  Factor 5: documentation_help (+0.684), support_responsiveness (+0.651), training_quality (+0.683)


# 8. Visualization

In [455]:
components = np.arange(1, len(eigenvalues_fa_unrotated) + 1)
fig = go.Figure()

fig.add_trace(go.Scatter(x= components,y= eigenvalues_fa_unrotated,name="Eigenvalues",mode='lines+markers'))
fig.add_trace(go.Scatter(x=components,y=[1.0]*len(eigenvalues_fa_unrotated),line=dict(color='red', width=4, dash='dash'),name="Kaiser Criterion",))

fig.update_layout(
        title={
            'text': "Factor Analysis Eigenvalues",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title='Factor Number (Component)',
        yaxis_title='Eigenvalue (Variance Explained)',
        hovermode="closest",
        xaxis=dict(tickmode='linear', tick0=1, dtick=1),
        template="plotly_white",
        width=800,
        height=500
    )

display(HTML(pio.to_html(fig, full_html=True, include_plotlyjs='cdn')))

In [456]:
x_labels = [factor_traductor["Factor"+str(i)] for i in range(n_factors)]
y_labels = var_names

fig1 = px.imshow(fa_rotated.loadings_.T,x = y_labels,y = x_labels,color_continuous_scale='RdBu',title="Varimax Rotated Factor Loadings")
display(HTML(pio.to_html(fig1, full_html=True, include_plotlyjs='cdn')))
fig2 = px.imshow(fa_unrotated.loadings_.T,x = y_labels,y = x_labels,color_continuous_scale='RdBu',title="Unrotated Factor Loadings")
display(HTML(pio.to_html(fig2, full_html=True, include_plotlyjs='cdn')))



# 9. Validation

In [457]:
colums = [factor_traductor["Factor"+str(i)] for i in range(n_factors)]
factor_scores_df_rotated = pd.DataFrame(fa_rotated.transform(variables_standarized),columns=colums)
factor_scores_df_unrotated = pd.DataFrame(fa_unrotated.transform(variables_standarized),columns=colums)

factor_scores_df_rotated.index = variables.index
factor_scores_df_unrotated.index = variables.index


c:\Users\dilos\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\dilos\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [458]:

analysis_df_rot = pd.concat([factor_scores_df_rotated, variables[outcome_vars]], axis=1)
analysis_df_unrot = pd.concat([factor_scores_df_unrotated, variables[outcome_vars]], axis=1)



In [465]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

htmls = []

for var in outcome_vars:
    model_formula = var+" ~ "+" + ".join(colums)


    model_rot = smf.ols(model_formula, data=analysis_df_rot).fit()
    model_unrot = smf.ols(model_formula, data=analysis_df_unrot).fit()

    print(var)
    print("Rotated:",model_rot.summary())
    print("Unrotated:",model_unrot.summary())

    coefficients_rot = model_rot.params[1:]
    factor_names_rot = coefficients_rot.index
    impact_values_rot = coefficients_rot.values

    # Sort by impact value to make the chart easy to read
    sorted_idx_rot = np.argsort(impact_values_rot)[::-1]
    impact_values_rot = impact_values_rot[sorted_idx_rot]
    factor_names_rot = factor_names_rot[sorted_idx_rot]

    coefficients_unrot = model_unrot.params[1:]
    factor_names_unrot = coefficients_unrot.index
    impact_values_unrot = coefficients_unrot.values

    # Sort by impact value to make the chart easy to read
    sorted_idx_unrot = np.argsort(impact_values_unrot)[::-1]
    impact_values_unrot = impact_values_unrot[sorted_idx_unrot]
    factor_names_unrot = factor_names_unrot[sorted_idx_unrot]

    # Create an interactive Plotly bar chart
    fig = go.Figure(data=[go.Bar(name="Varimax Rotated",
        x=impact_values_rot,
        y=factor_names_rot,
        orientation='h', # Horizontal bars
        marker_color=['green' if c > 0 else 'red' for c in impact_values_rot] # Color bars by positive/negative impact
    ),go.Bar(name="No rotation",
        x=impact_values_unrot,
        y=factor_names_unrot,
        orientation='h', # Horizontal bars
        marker_color=['Blue' if c > 0 else 'violet' for c in impact_values_unrot] # Color bars by positive/negative impact
    )])

    fig.update_layout(
        title=f'Business Impact of Factors on {var.replace("_", " ").title()}',
        xaxis_title='Coefficient Value (Standardized Impact)',
        yaxis_title='Factor Name',template="plotly_white",width=800,height=500)
    
    htmls.append(HTML(pio.to_html(fig, full_html=True, include_plotlyjs='cdn')))
    

overall_satisfaction
Rotated:                              OLS Regression Results                             
Dep. Variable:     overall_satisfaction   R-squared:                       0.604
Model:                              OLS   Adj. R-squared:                  0.603
Method:                   Least Squares   F-statistic:                     983.1
Date:                  Thu, 13 Nov 2025   Prob (F-statistic):               0.00
Time:                          22:13:27   Log-Likelihood:                -2458.9
No. Observations:                  3235   AIC:                             4930.
Df Residuals:                      3229   BIC:                             4966.
Df Model:                             5                                         
Covariance Type:              nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------

In [460]:
df_melted = factor_scores_df_unrotated.melt(var_name='Factor', value_name='Score')


fig = px.histogram(df_melted, x="Score", color="Factor", nbins=40, facet_col="Factor", 
        facet_col_wrap=3,title="Distribution of Standardized Factor Scores (Mean=0, SD=1)",
        template="plotly_white",hover_data={"Score": ":.2f"})


fig.update_layout(
    height=800, 
    width=1000,
    showlegend=False
)


for axis in fig.layout:
    if isinstance(fig.layout[axis], go.layout.XAxis): #
        fig.add_vline(x=0,line_width=1,line_dash="dash", line_color="red", 
                        annotation_text='Mean=0', annotation_position='top right')
        

fig.update_xaxes(title_text='Standardized Score')
fig.update_yaxes(title_text='Frequency (Customer Count)')

display(HTML(pio.to_html(fig, full_html=True, include_plotlyjs='cdn')))

In [466]:
for figure in htmls:
    display(figure)

**Team:** Equipo 8

**Members:**
- Diego Lopez Schmill (A01666290) - Data exploration and factor extraction
- Diego Lopez Schmill (A01666290) - Factor interpretation and business insights
- Juan Pablo Arteaga Patiño (A01665795) - Visualization and recommendations

**Deliverable Links:**
- **Presentation:** [\[Available on Google Sliders\]](https://docs.google.com/presentation/d/1p7n9nkhL8QJG1upwAWcABLtZFJGT01THpj1WxDIZ-1E/edit?usp=sharing)
- **Executive Summary:** [\[Available on Google Docs\]](https://docs.google.com/document/d/1bZhoU6Jm0NrBT8b0Mu1ItrFnD9sGLrwt22FJG3HZHWo/edit?usp=sharing)
- **Dataset:** `customer_satisfaction_data.csv`

**Completion Date:** 13/11/2025